In [22]:
import torch
import numpy as np
from math import ceil
import torch.nn as nn
import torch.nn.functional as F
from typing import Tuple
from torch.utils.data import DataLoader
from torchvision.datasets import DatasetFolder

## Loading the baseline model

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=(10,), stride=(1,))
        self.fc1 = nn.Linear(65496, 1)  # Adjust the input size based on your data size

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        x = x.view(-1, 65496)
        return F.sigmoid(self.fc1(x))

In [4]:
model = ConvNet().to(device)
model.load_state_dict(torch.load("model.pt"))
model

ConvNet(
  (conv1): Conv1d(1, 16, kernel_size=(10,), stride=(1,))
  (fc1): Linear(in_features=65496, out_features=1, bias=True)
)

### Evaluation

In [5]:
class BinaryTransform:
    def __init__(self, input_length):
        self.input_length = input_length

    def __call__(self, binary_data):
        binary_data = np.frombuffer(binary_data, dtype=np.uint8)
        
        l = len(binary_data)

        # Pad or truncate the binary data
        if l < self.input_length:
            padding = np.zeros(self.input_length - l, dtype=np.uint8)
            binary_data = np.concatenate((binary_data, padding))
        elif l > self.input_length:
            excess = ceil(l / self.input_length)
            padding = np.zeros(self.input_length * excess - l, dtype=np.uint8)
            binary_data = np.concatenate((binary_data, padding))
            binary_data = binary_data.reshape(len(binary_data)//excess, -1)
            binary_data = np.mean(binary_data, axis=1)
            
        # Scale the data to [0, 1]
        scaled_data = binary_data / 255.0
        tensor = torch.tensor(scaled_data, dtype=torch.float32)
        return tensor.unsqueeze(0)

In [6]:
test_data_path = "data/test"
transform = BinaryTransform(16384)
test_dataset = DatasetFolder(
    root=test_data_path,
    loader=lambda x: open(x, 'rb').read(),
    extensions=('',),
    transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=64)

In [15]:
correct = 0
total = len(test_dataset)

with torch.no_grad():
    for X, y in test_loader:
        X, y = X.to(device), y.to(device)
        y_pred = model(X).squeeze()
        pred_label = y_pred > 0.5
        correct += pred_label.eq(y).sum().item()

correct / total

0.9923076923076923

## Baseline adversarial accuracy with random suffix

In [40]:
class BinaryTransformWithMask:
    def __init__(self, input_length: int, adversarial_ratio: float) -> None:
        self.input_length = input_length
        self.adversarial_ratio = adversarial_ratio

    def __call__(self, binary_data: bytes) -> Tuple[torch.Tensor, np.array]:
        """Returns the model input prepared as a (1,input_length) Tensor,
        and the mask which indicates positions influenced exclusively by the adv. suffix."""
        l_original = len(binary_data)
        binary_array = self.get_extended_binary_array(binary_data)
        l_with_adversarial = len(binary_array)


        if l_with_adversarial < self.input_length:
            # the bytes array is too short and zero padding should be added to match input_length
            # the mask does not include the zero-padding bytes
            padding = np.zeros(self.input_length - l_with_adversarial, dtype=np.uint8)
            binary_array = np.concatenate((binary_array, padding))
            mask = np.arange(l_original, l_with_adversarial)
        elif l_with_adversarial > self.input_length:
            # the byte array should be split into ceil(l_with_adversarial / input_length) chunks,
            # with the last chunk being padded to chunk size if needed
            # when padding is used, that last chunk is not considered part of the adversarial mask 
            window_size = ceil(l_with_adversarial / self.input_length)
            num_original_groups = ceil(l_original / window_size) # byte groups influenced by the original binary
            l_padding = self.input_length * window_size - l_with_adversarial
            num_padding_groups = ceil(l_padding / window_size)
            padding = np.zeros(l_padding, dtype=np.uint8)
            binary_array = np.concatenate((binary_array, padding))
            binary_array = binary_array.reshape(-1, window_size)
            binary_array = np.mean(binary_array, axis=1)
            mask = np.arange(num_original_groups, self.input_length - num_padding_groups)
            
        # Scale the data to [0, 1]
        scaled_data = binary_array / 255.0
        tensor = torch.tensor(scaled_data, dtype=torch.float32)
        return tensor.unsqueeze(0), mask

    def get_extended_binary_array(self, binary_data: bytes) -> np.array:
        """Build the extended binary with the adversarial suffix set to zero."""
        l = len(binary_data)
        l_with_adversarial = ceil(l * (1 + self.adversarial_ratio))
        binary_array = np.zeros(l_with_adversarial, dtype=np.uint8)
        binary_array[:l] = np.frombuffer(binary_data, dtype=np.uint8)
        return binary_array

In [42]:
# example #1 in the assignment text
binary_data = bytes(list(range(1, 11)))
transform = BinaryTransformWithMask(input_length=6, adversarial_ratio=0.4) # +4 bytes
X, M = transform(binary_data)
X, M

(tensor([[0.0078, 0.0196, 0.0314, 0.0131, 0.0000, 0.0000]]),
 array([], dtype=int64))

In [43]:
# example #2 in the assignment text
binary_data = bytes(list(range(1, 11)))
transform = BinaryTransformWithMask(input_length=6, adversarial_ratio=0.5) # +5 bytes
X, M = transform(binary_data)
X, M

(tensor([[0.0078, 0.0196, 0.0314, 0.0131, 0.0000, 0.0000]]), array([4]))

In [71]:
file_path = "data/victim/malware/0d41d1d904aecf716303f55108e020fbd9a4dbcd997efb08fba5e10e936d419c"
with open(file_path, "rb") as f:
    binary_data = f.read()

transform = BinaryTransformWithMask(input_length=2**14, adversarial_ratio=0.05)
input_tensor, M = transform(binary_data)
input_tensor = input_tensor.unsqueeze(0) # add batch dimension

In [72]:
model(input_tensor)

tensor([[0.9895]], grad_fn=<SigmoidBackward0>)

In [110]:
adversary_features = torch.zeros(len(M), dtype=torch.float32, requires_grad=True)
opt = torch.optim.SGD([adversary_features], lr=0.01, weight_decay=0.1)

loss_fn = nn.BCELoss()

for t in range(1000):
    input_with_adversary = input_tensor.clone()
    input_with_adversary[...,M] += adversary_features
    pred = model(input_with_adversary).squeeze()
    loss = -loss_fn(pred, torch.tensor(1, dtype=torch.float32)) # 1 = malware
    if t % 5 == 0:
        print(t, loss.item())
       
    opt.zero_grad()
    loss.backward()
    adversary_features.grad.sign_()
    opt.step()

    # projection with clipping
    adversary_features.data.clamp_(0, 1)

0 -0.010592754930257797
5 -0.012299768626689911
10 -0.015437961556017399
15 -0.01954956166446209
20 -0.02464163675904274
25 -0.03102256916463375
30 -0.03826124221086502
35 -0.04707461595535278
40 -0.0577281154692173
45 -0.07001614570617676
50 -0.08530186116695404
55 -0.10434142500162125
60 -0.12705855071544647
65 -0.15378989279270172
70 -0.18580514192581177
75 -0.22289367020130157
80 -0.26724955439567566
85 -0.3175199031829834
90 -0.37661269307136536
95 -0.4436857998371124
100 -0.5188723206520081
105 -0.6033344864845276
110 -0.6236594319343567
115 -0.6314519643783569
120 -0.6379726529121399
125 -0.6432398557662964
130 -0.6468443870544434
135 -0.6507037878036499
140 -0.6554455161094666
145 -0.6585882306098938
150 -0.6615903377532959
155 -0.66433185338974
160 -0.6673728823661804
165 -0.6690101027488708
170 -0.6722534894943237
175 -0.6722065210342407
180 -0.6751646399497986
185 -0.6743672490119934
190 -0.6770105361938477
195 -0.6776541471481323
200 -0.6791898012161255
205 -0.6796035766601

In [111]:
model(input_with_adversary)

tensor([[0.4985]], grad_fn=<SigmoidBackward0>)

In [112]:
adversary_features

tensor([0.0000, 1.0000, 0.0000, 1.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000,
        1.0000, 0.0000, 0.4762, 0.6769, 0.0000, 1.0000, 1.0000, 0.0000, 1.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000,
        1.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000,
        1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000, 0.8556,
        1.0000, 0.2929, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.8423, 0.0000,
        0.0000, 0.0539, 0.5758, 0.2166, 0.1887, 0.0000, 0.0000, 0.0000, 0.0568,
        0.1886, 0.0000, 0.0654, 1.0000, 0.0000, 0.0000, 0.3308, 0.0960, 0.0000,
        0.2714, 0.0000, 0.0000, 0.0000, 0.4218, 0.0068, 0.0100, 0.0000, 0.0850,
        0.0000, 0.1037, 0.0100, 0.0000, 0.0157, 0.0808, 0.0820, 0.0736, 0.0000,
        0.0000, 0.0000, 0.4227, 0.1191, 0.0224, 0.0405, 0.1521, 0.9105, 0.0000,
        0.4200, 0.2129, 0.0000, 0.1269, 